In [1]:
import cv2
import os
import numpy as np

OUTPUT_SIZE = (450, 635)
OUTPUT_IMAGE_DIR = "./images"
INPUT_VIDEO_DIR = "./videos"
NUM_OF_IMAGE_PER_CARD = 50

In [2]:
def process_image(image):
    _, w, _ = image.shape
    cropped = image[:, w // 4:(3 * w) // 4]

    blur = cv2.GaussianBlur(cropped,(5,5), 0)

    grayscale = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)

    _, threshold = cv2.threshold(grayscale, 100, 255, cv2.THRESH_BINARY)

    x,y,w,h = cv2.boundingRect(threshold)

    cropped = cropped[y:y+h, x:x+w]

    rescaled = cv2.resize(cropped, OUTPUT_SIZE)

    return rescaled

In [3]:
def extract_frames(video_path, output_path, num_of_images=NUM_OF_IMAGE_PER_CARD):
    os.makedirs(output_path, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    
    number_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    random_frames = np.sort(np.random.choice(number_of_frames, num_of_images, replace=False))
    
    frame_count = 0
    saved_frames = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count in random_frames:
            processed_frame = process_image(frame)
            frame_filename = os.path.join(output_path, f"card_{saved_frames:02d}.png")
            cv2.imwrite(frame_filename, processed_frame)
            saved_frames += 1

        frame_count += 1
    
    cap.release()

In [4]:
for decks_dir in os.listdir(INPUT_VIDEO_DIR):
    decks_path = os.path.join(INPUT_VIDEO_DIR, decks_dir)
    for video_file in os.listdir(decks_path):
        video_path = os.path.join(INPUT_VIDEO_DIR, os.path.join(decks_dir, video_file))
        output_path = os.path.join(OUTPUT_IMAGE_DIR, os.path.join(decks_dir, video_file))
        extract_frames(video_path, output_path)